# Quick HDF5 benchmarks
from https://cyrille.rossant.net/moving-away-hdf5/

We compare the performance of reading a subset of a large array:
* in memory with NumPy
* with h5py
* with memmap using an HDF5 file
* with memmap using an NPY file

This illustrates our performance issues with HDF5 in our very particular use case (accessing a small number of lines in a large "vertical" rectangular array).

In [1]:
import h5py
import numpy as np

We'll use this function to bypass the slow h5py data access with a faster memory mapping (only works on uncompressed contiguous datasets):

In [2]:
def _mmap_h5(path, h5path):
    with h5py.File(path) as f:
        ds = f[h5path]
        # We get the dataset address in the HDF5 fiel.
        offset = ds.id.get_offset()
        # We ensure we have a non-compressed contiguous array.
        assert ds.chunks is None
        assert ds.compression is None
        assert offset > 0
        dtype = ds.dtype
        shape = ds.shape
    arr = np.memmap(path, mode='r', shape=shape, offset=offset, dtype=dtype)
    return arr

We read the files to compare:

In [3]:
fname_nozipped = '/remote/ceph/group/magic/MAGIC-LST/ctapipe/CrabNebula/Calibrated/2018_03_09/20180309_05070968_Y_CrabNebula-W0.40+215.hdf5'

f_nozipped = h5py.File(fname_nozipped, 'r')
f_zipped = h5py.File('/remote/ceph/group/magic/MAGIC-LST/ctapipe/CrabNebula/Calibrated/2018_03_09/20180309_05070968_Y_CrabNebula-W0.40+215.gz.hdf5', 'r')

image = f_nozipped['/dl1/tel1/image'][:]
#f_nozipped.close()
#print(arr.shape)

Write numpy array to comparison

In [4]:
np.save('test.npy', image)

In [6]:
%ls -lrth

total 1.6G
-rw-r--r-- 1 mhuetten magic 879K Oct  2 09:01 star2hdf5_checkpointing.ipynb
-rw-r--r-- 1 mhuetten magic  16K Nov  6 10:45 merpp2hdf5.ipynb
-rw-r--r-- 1 mhuetten magic  42K Nov  6 10:45 sorcerer2hdf5_deprecated.ipynb
-rwxr-xr-x 1 mhuetten magic 7.9K Nov  6 11:36 hdf5-benchmark.ipynb*
-rw-r--r-- 1 mhuetten magic  39K Nov  6 12:29 sorcerer2hdf5.py
-rw-r--r-- 1 mhuetten magic 1.6G Nov  6 16:11 test.npy


## Slices

In [9]:
ind = 12654

In [10]:
print('in memory')
%timeit image[ind] * 1
print()
print('h5py')
%timeit f_nozipped['/dl1/tel1/image'][ind] * 1
print()
print('h5py compressed')
%timeit f_zipped['/dl1/tel1/image'][ind] * 1
print()
print('memmap of uncompressed HDF5 file')
%timeit _mmap_h5(fname_nozipped, '/dl1/tel1/image')[ind] * 1
print()
print('memmap of NPY file')
%timeit np.load('test.npy')[ind] * 1

in memory
1.25 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

h5py
222 µs ± 3.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

h5py compressed
43 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

memmap of uncompressed HDF5 file
1.51 ms ± 54.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

memmap of NPY file
540 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Fancy indexing

Fancy indexing is what we have to use in our particular use-case.

In [13]:
ind = np.unique(np.random.randint(0, n, n // 100))

In [15]:
len(ind)

999

In [16]:
print('in memory')
%timeit arr[ind, :] * 1
print()
print('h5py')
%timeit f['/test'][ind, :] * 1
print()
print('memmap of HDF5 file')
%timeit _mmap_h5('test.h5', '/test')[ind, :] * 1
print()
print('memmap of NPY file')
%timeit np.load('test.npy', mmap_mode='r')[ind, :] * 1

in memory
100 loops, best of 3: 2.05 ms per loop

h5py
10 loops, best of 3: 53.3 ms per loop

memmap of HDF5 file
100 loops, best of 3: 5.62 ms per loop

memmap of NPY file
100 loops, best of 3: 5.12 ms per loop


Note that h5py uses a [slow algorithm for fancy indexing](https://gist.github.com/rossant/7b4704e8caeb8f173084#gistcomment-1665072), so HDF5 is not the only cause of the slowdown.